In [8]:
import numpy as np
import pandas as pd
from monke import functions as fct

data = pd.read_csv("magneton.csv").iloc[:2]
magneton = data["Magneton"].mean()
magneton_err = data["Magneton"].std() + data["Magneton Fehler"].mean()
print(fct.error_round(magneton*1e24, magneton_err*1e24,))

('9.2', '1.2')


# Auflösungsvermögen + Finesse

In [27]:
import pandas as pd
import numpy as np
from callibration import do_calibration
from typing import Callable
from monke import constants as ct
from monke import functions as fct

r = 0.85**2
wavelength = 644e-9
d = 4e-3
n = 1.457
mu = 9.274e-24

finesse = np.pi * np.sqrt(r) / (1 - r)
a = 2*n*d/wavelength * finesse

print(f"finesse = {finesse}, A = {a}")

data: pd.DataFrame = pd.DataFrame(np.loadtxt("../Data/Zeeman/kalibrierung_3.txt", skiprows=1),
                                             columns=["I / A", "B / mT"]).query("`I / A` < 9.5")
_, _, bfield, _ = do_calibration(data, sy=1) # in mT

i1 = (3.6, .1)
i2 = (1.3, .2)
a1 = ct.h*ct.c/(wavelength*mu*bfield(i1[0]) / 1000)
a1_err = a1 * i1[1] / i1[0]
print(f"auflösungsvermögen transversal: {functions.error_round(a1, a1_err)}")

a2 = ct.h*ct.c/(wavelength*mu*bfield(i2[0]) / 1000)
a2_err = a2 * i2[1] / i2[0]
a2 /= 2
a2_err /= 2

print(f"auflösungsvermögen transversal: {functions.error_round(a2, a2_err, "scientific")}")

finesse1 = a1*wavelength / (2*n*d), a1_err*wavelength / (2*n*d)
finesse2 = a2*wavelength / (2*n*d), a2_err*wavelength / (2*n*d)
print(f"finessee trans: {fct.error_round(*finesse1)}")
print(f"finessee longitudinal: {fct.error_round(*finesse2)}")





finesse = 9.622896416401165, A = 174168.44818256522


/home/gabriel/anaconda3/envs/p4_312/lib/python3.12/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)


auflösungsvermögen transversal: ('137000', '4000')
auflösungsvermögen transversal: ('1.90000(30000)e5', 5)
finessee trans: ('7.5', '0.3')
finessee longitudinal: ('10.4', '1.7')


## Dopplerverbreiterung

In [24]:
t = 1000
m = 112.4 * ct.u
delta_l = wavelength / ct.c *np.sqrt(t*8*ct.kb*np.log(2) / m)
print(f"Delta lambda = {delta_l*1e12} pm")


Delta lambda = 1.3758048162472396 pm
